# Hyperparameter tuning by randomized-search

No bloco de notas anterior, mostramos como usar uma abordagem de pesquisa em grade para pesquisar os melhores hiperparâmetros, maximizando o desempenho estatístico de um modelo preditivo.

No entanto, uma abordagem de pesquisa em grade tem limitações. Ele não aumenta quando o número de parâmetros a serem ajustados está aumentando. Além disso, a grade irá impor uma regularidade durante a pesquisa que pode ser problemática.

Neste caderno, apresentaremos outro método para ajustar hiperparâmetros, denominado pesquisa aleatória.

## Nosso modelo preditivo

Vamos recarregar o conjunto de dados como fizemos anteriormente:

In [1]:
from sklearn import set_config

set_config(display="diagram")

In [2]:
import pandas as pd

adult_census = pd.read_csv("adult-census.csv")
# drop the duplicated column `"education-num"` as stated in the first notebook
adult_census = adult_census.drop(columns=['ID','fnlwgt:','education-num:'])

Extraímos a coluna que contém o destino.

In [3]:
target_name = "class"
target = adult_census[target_name]
target

0        <=50K
1        <=50K
2        <=50K
3        <=50K
4        <=50K
         ...  
32556    <=50K
32557     >50K
32558    <=50K
32559    <=50K
32560     >50K
Name: class, Length: 32561, dtype: object

Tiramos de nossos dados o alvo e a coluna `" education-num "` que
duplica a informação com colunas `" educação "`.

In [10]:
data = adult_census.drop(columns=[target_name])
data['native-country:'] = data['native-country:'].replace(['Holand-Netherlands'], 'United-States')
data.head()

,age,workclass,education:,marital-status:,occupation:,relationship:,race:,sex:,capital-gain:,capital-loss:,hours-per-week:,native-country:
0,39,State-gov,Bachelors,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States
1,50,Self-emp-not-inc,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States
2,38,Private,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States
3,53,Private,11th,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States
4,28,Private,Bachelors,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba


Depois que o conjunto de dados é carregado, nós o dividimos em conjuntos de treinamento e teste.

In [11]:
from sklearn.model_selection import train_test_split

data_train, data_test, target_train, target_test = train_test_split(
    data, target, random_state=42)

Vamos criar o mesmo pipeline preditivo visto no grid-search
seção.

In [12]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import make_column_selector as selector

categorical_columns_selector = selector(dtype_include=object)
categorical_columns = categorical_columns_selector(data)

categorical_preprocessor = OrdinalEncoder()

preprocessor = ColumnTransformer([
    ('cat-preprocessor', categorical_preprocessor, categorical_columns)],
    remainder='passthrough', sparse_threshold=0)
preprocessor

ColumnTransformer(remainder='passthrough', sparse_threshold=0,
                  transformers=[('cat-preprocessor', OrdinalEncoder(),
                                 ['workclass', 'education:', 'marital-status:',
                                  'occupation:', 'relationship:', 'race:',
                                  'sex:', 'native-country:'])])

In [13]:
# for the moment this line is required to import HistGradientBoostingClassifier
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.pipeline import Pipeline

model = Pipeline([
    ("preprocessor", preprocessor),
    ("classifier",
     HistGradientBoostingClassifier(random_state=42, max_leaf_nodes=4))])
model

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough', sparse_threshold=0,
                                   transformers=[('cat-preprocessor',
                                                  OrdinalEncoder(),
                                                  ['workclass', 'education:',
                                                   'marital-status:',
                                                   'occupation:',
                                                   'relationship:', 'race:',
                                                   'sex:',
                                                   'native-country:'])])),
                ('classifier',
                 HistGradientBoostingClassifier(max_leaf_nodes=4,
                                                random_state=42))])

## Tuning usando uma busca aleatória

Com o estimador `GridSearchCV`, os parâmetros precisam ser especificados
explicitamente. Já mencionamos que explorar um grande número de valores para
parâmetros diferentes serão rapidamente intratáveis.

Em vez disso, podemos gerar aleatoriamente os candidatos a parâmetros. De fato,
tal abordagem evita a regularidade da grade. Portanto, adicionar mais
avaliações podem aumentar a resolução em cada direção. Isto é o
caso na situação frequente em que a escolha de alguns hiperparâmetros
não é muito importante, como para o hiperparâmetro 2 na figura abaixo.

![Randomized vs grid search](imagens/hyper.png)

Na verdade, o número de pontos de avaliação precisa ser dividido entre os
dois hiperparâmetros diferentes. Com uma grade, o perigo é que o
região de bons hiperparâmetros caem entre a linha da grade: este
região está alinhada com a grade, dado que o hiperparâmetro 2 tem um fraco
influência. Em vez disso, a pesquisa estocástica terá uma amostra do hiperparâmetro 1
independentemente do hiperparâmetro 2 e encontre a região ideal.

A classe `RandomizedSearchCV` permite essa pesquisa estocástica. Isto é
usado de forma semelhante ao `GridSearchCV`, mas as distribuições de amostragem
precisam ser especificados em vez dos valores dos parâmetros. Por exemplo, nós
irá desenhar candidatos usando uma distribuição log-uniforme porque os parâmetros
estamos interessados em assumir valores positivos com uma escala de log natural (.1 é tão próximo de 1 quanto 10).

<div class="admonition note alert alert-info">
<p class="first admonition-title" style="font-weight: bold;">Note</p>
<p class="last">Random search (with <tt class="docutils literal">RandomizedSearchCV</tt>) é tipicamente benéfico comparado
para pesquisa em grade (com <tt class="docutils literal">GridSearchCV</tt>) para otimizar 3 ou mais hyperparameters.</p>
</div>

Vamos otimizar 3 outros parâmetros além daqueles que otimizado acima:

* `max_iter`: corresponde ao número de árvores no conjunto;
* `min_samples_leaf`: corresponde ao número mínimo de amostras
  exigido em uma folha;
* `max_bins`: corresponde ao número máximo de caixas para construir o
  histogramas.
  
<div class="admonition note alert alert-info">
<p class="first admonition-title" style="font-weight: bold;">Note</p>
<p class="last">O <tt class="docutils literal">loguniform</tt> função de SciPy retorna um número flutuante. Desde que nós
queremos que essa distribuição crie um inteiro, vamos criar uma classe que
irá converter o número flutuante em um inteiro.</p>
</div>  

In [14]:
from scipy.stats import loguniform


class loguniform_int:
    """Integer valued version of the log-uniform distribution"""
    def __init__(self, a, b):
        self._distribution = loguniform(a, b)

    def rvs(self, *args, **kwargs):
        """Random variable sample"""
        return self._distribution.rvs(*args, **kwargs).astype(int)

In [15]:
from sklearn.model_selection import RandomizedSearchCV

param_distributions = {
    'classifier__l2_regularization': loguniform(1e-6, 1e3),
    'classifier__learning_rate': loguniform(0.001, 10),
    'classifier__max_leaf_nodes': loguniform_int(2, 256),
    'classifier__min_samples_leaf': loguniform_int(1, 100),
    'classifier__max_bins': loguniform_int(2, 255)}

model_random_search = RandomizedSearchCV(
    model, param_distributions=param_distributions, n_iter=10,
    n_jobs=4, cv=5)
model_random_search.fit(data_train, target_train)

RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('preprocessor',
                                              ColumnTransformer(remainder='passthrough',
                                                                sparse_threshold=0,
                                                                transformers=[('cat-preprocessor',
                                                                               OrdinalEncoder(),
                                                                               ['workclass',
                                                                                'education:',
                                                                                'marital-status:',
                                                                                'occupation:',
                                                                                'relationship:',
                                                                                'race:',
                                                                                'sex:',
                                                                                'native-country:'])])),
                                             ('classifier',
                                              HistGradientBoostingClassifier(max_leaf_nodes=...
                   param_distributions={'classifier__l2_regularization': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000029AC6B09A90>,
                                        'classifier__learning_rate': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000029AC6BA2580>,
                                        'classifier__max_bins': <__main__.loguniform_int object at 0x0000029AC6BA2E80>,
                                        'classifier__max_leaf_nodes': <__main__.loguniform_int object at 0x0000029AC6B094F0>,
                                        'classifier__min_samples_leaf': <__main__.loguniform_int object at 0x0000029AC6B09400>})

Em seguida, podemos calcular a pontuação de precisão no conjunto de teste.

In [16]:
accuracy = model_random_search.score(data_test, target_test)

print(f"The test accuracy score of the best model is " f"{accuracy:.2f}")

The test accuracy score of the best model is 0.86


In [17]:
from pprint import pprint

print("The best parameters are:")
pprint(model_random_search.best_params_)

The best parameters are:
{'classifier__l2_regularization': 0.002610152821950518,
 'classifier__learning_rate': 0.19831644733715803,
 'classifier__max_bins': 50,
 'classifier__max_leaf_nodes': 3,
 'classifier__min_samples_leaf': 1}


Podemos inspecionar os resultados usando os atributos `cv_results` como anteriormente
fez.

In [18]:
def shorten_param(param_name):
    if "__" in param_name:
        return param_name.rsplit("__", 1)[1]
    return param_name

In [19]:
# get the parameter names
column_results = [
    f"param_{name}" for name in param_distributions.keys()]
column_results += [
    "mean_test_score", "std_test_score", "rank_test_score"]

cv_results = pd.DataFrame(model_random_search.cv_results_)
cv_results = cv_results[column_results].sort_values(
    "mean_test_score", ascending=False)
cv_results = cv_results.rename(shorten_param, axis=1)
cv_results

,l2_regularization,learning_rate,max_leaf_nodes,min_samples_leaf,max_bins,mean_test_score,std_test_score,rank_test_score
2,0.00261015,0.198316,3,1,50,0.852785,0.003919,1
8,0.375509,0.3619,227,12,88,0.852211,0.005421,2
7,0.114127,0.0122273,75,36,13,0.829934,0.002693,3
9,389.578,0.587338,6,33,4,0.813718,0.001792,4
3,0.00546389,0.315606,61,35,4,0.810565,0.001675,5
4,2.25488e-06,0.106158,24,12,3,0.807535,0.005701,6
0,6.91433,0.0194886,6,4,10,0.806306,0.002541,7
1,0.0219691,0.0057492,10,22,6,0.757821,0.000082,8
5,22.8969,0.00408988,12,14,30,0.757821,0.000082,8
6,0.691669,0.00492552,9,13,3,0.757821,0.000082,8


Na prática, uma pesquisa aleatória de hiperparâmetros geralmente é executada com um grande
número de iterações. A fim de evitar o custo de computação e ainda fazer um
análise decente, carregamos os resultados obtidos de uma pesquisa semelhante com 200
iterações.

* [`clique para ver o plot`](https://inria.github.io/scikit-learn-mooc/python_scripts/parameter_tuning_randomized_search.html)